# Overview

This notebook is meant to help show how some of the lower level classes and objects from our code base work if you do not wish to use the experiment setup in `src/experiments`

In all of these examples I assume you have the model checkpoints in your `{PROJECT_ROOT}/trained_models` folder.  All of these example generations were generated using T5-Large step models.  T5 large is extremely good for debugging and showing example use cases, but we found it to have poor performance.  In our paper we use T5 3B.

Also, all of this was ran with transformers==4.20.0 -- other versions may cause problems.

# Trees!

In [48]:
from search.tree import Tree, StepGeneration

goal = 'The Sun is Earth\'s star'
premises = ['The Earth is a planet.', 'Planets revolve around their star.', 'The Earth revolves around the Sun.']

# Example of gold annotations or maybe generations from a model
# p0 means premise in index 0 of the premise list, i0 means intermediate 0 in the list of intermediates.
intermediates = [StepGeneration(['p0', 'p1'], output="The Earth revolves around it's star"), StepGeneration(['i0', 'p2'], output=goal)]

# Tree's are the basic primitive used in the search algorithm in our system.  Although nearly all models can work with bare strings, a Tree is used to keep track of the search space.
tree = Tree(goal, premises, hypotheses=[], intermediates=intermediates)

# Step Models!

Here we show how to call a deductive and abductive step model (they behave almost entirely the same when sampling from them directly).

In [49]:
from search.step_type import ForwardStepModel

deductive_model = ForwardStepModel('t5_large_pps_eb_step')

deductive_model.sample('The planet Earth revolves around the Sun. Planets revolve around their star.')

['Earth revolves around its star.']

In [50]:
from search.step_type import AbductiveStepModel

abductive_model = AbductiveStepModel('t5_abductive_step')

abductive_model.sample('The planet Earth revolves around the Sun. Earth revolves around its star.')

['The Sun is a kind of star.']

To sample multiple generations for a single call you can use sample_override per sample call (you can also specify the number of returned generations in the constructor of the class)

In [51]:
abductive_model.sample('The planet Earth revolves around the Sun. Earth revolves around its star.', sample_override=20)

['The Sun is a kind of star.',
 'The Sun is a kind of star.',
 'The Sun is the star located on the planet Earth.',
 'The sun is a kind of star.',
 'The Sun is a kind of star.',
 'The Sun is a kind of star.',
 'Sun is a kind of star.',
 'The Sun is a kind of star.',
 'The Sun is a kind of star.',
 'The sun is a kind of star.',
 'The sun is a kind of star.',
 'The Sun is a kind of star.',
 'Sun is a kind of star.',
 'The sun is a kind of star.',
 'The sun is the star behind the planet Earth.',
 "The sun is the sun's star.",
 'The Sun is a kind of star.',
 'The Sun is a kind of star.',
 'The Sun is the star that is located in the center of the earth.',
 'The Sun is a kind of star.']

Here's an example of calling a step model with a tree

In [52]:
formatted_input = deductive_model.format(tree, intermediates[0].inputs)
tree_deductive_outputs = deductive_model.sample(formatted_input, sample_override=20)

[f'GOLD: {intermediates[0].output}', '', *tree_deductive_outputs]

["GOLD: The Earth revolves around it's star",
 '',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around their star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.',
 'Earth revolves around its star.',
 'The Earth revolves around its star.',
 'The Earth revolves around its star.']

# Entailment Models!

In [53]:
from search.entailment.entailment_model import EntailmentModel

entailment_model = EntailmentModel('wanli_entailment_model')

# The entailment score returns the probability of the Entailment class being true (vs contradiction or neutral classes).  The inputs must be of equal length, think of the returned list as, list(ENTAIL(targets[0], predictions[0])... ENTAIL(targets[n], predictions[n])
scores = entailment_model.score([intermediates[0].output]*len(tree_deductive_outputs), tree_deductive_outputs)

[f'GOLD: {intermediates[0].output}', '', *[f'{score:.4f}: {gen}' for gen, score in zip(tree_deductive_outputs, scores)]]

  0%|          | 0/1 [00:00<?, ?ba/s]

["GOLD: The Earth revolves around it's star",
 '',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.6412: The Earth revolves around their star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8797: Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8797: Earth revolves around its star.',
 '0.8954: The Earth revolves around its star.',
 '0.8954: The Earth revolves around its sta

# Heuristic Models!

This examples slightly complicated (because we show multiple things at once) -- you can create Steps manually, but this is trying to show the way we intended from the search algorithm.

In [65]:
from search.heuristics.heuristics.steptype_modeled import StepTypeModeled
from search.step_type import ForwardStepType
from search.fringe.fringe_item import Step

# Most of the time in our search aglorithm, everything is processed in "steps" where a step is a single input to a step model that can generate multiple new step generations.  These step generations are then made into new ranked Steps and added to a fringe.  More info can be seen in `src/search/search.py`
# The lifecycle of a "step" in the search process is tethered to a step model (each step type, forward or abductive, have their own rules about how steps can be made)

# Set up a deductive step type, responsible for making string generations into step_generations
deductive_step_type = ForwardStepType(deductive_model)
# Manually create the step we took earlier in this notebook
step = Step(['p0', 'p1'], deductive_step_type)
# Create the step generations
_, _, new_step_gens = deductive_step_type.generation_to_step_generation(generations=tree_deductive_outputs, step=step)


# This is just ensuring you haven't added the new intermediates before generating new steps -- notebook stuff since you can re-run things or run them out of order, not completely necessary for real code (though don't try to create new steps with intermediates/hypotheses/premises you already have added to a Tree or it'll be weird)
tree.intermediates = intermediates


# Generate all the new possible steps that can be taken given the new steps
new_steps = deductive_step_type.generate_steps(tree, [], [], new_intermediates=new_step_gens)
tree.intermediates = [*intermediates, *new_step_gens]
# Create a heuristic model
heuristic = StepTypeModeled(forward_name='forward_v3_gc', abductive_name='abductive_gc')
# Score all the new steps.
step_scores = heuristic.score_steps(tree, new_steps)

[f'{score:.4f} : {step.inputs}' for step, score in zip(new_steps, step_scores)]

["3.2884 : ['i2', 'i0']",
 "3.3076 : ['i0', 'i2']",
 "-0.6562 : ['i2', 'i1']",
 "0.8556 : ['i1', 'i2']",
 "3.3191 : ['i2', 'i3']",
 "3.3191 : ['i3', 'i2']",
 "3.3191 : ['i2', 'i4']",
 "3.3191 : ['i4', 'i2']",
 "3.3191 : ['i2', 'i5']",
 "3.3191 : ['i5', 'i2']",
 "3.3136 : ['i2', 'i6']",
 "3.3015 : ['i6', 'i2']",
 "3.3191 : ['i2', 'i7']",
 "3.3191 : ['i7', 'i2']",
 "3.3191 : ['i2', 'i8']",
 "3.3191 : ['i8', 'i2']",
 "3.3191 : ['i2', 'i9']",
 "3.3191 : ['i9', 'i2']",
 "3.3268 : ['i2', 'i10']",
 "3.3350 : ['i10', 'i2']",
 "3.3191 : ['i2', 'i11']",
 "3.3191 : ['i11', 'i2']",
 "3.3191 : ['i2', 'i12']",
 "3.3191 : ['i12', 'i2']",
 "3.3191 : ['i2', 'i13']",
 "3.3191 : ['i13', 'i2']",
 "3.3191 : ['i2', 'i14']",
 "3.3191 : ['i14', 'i2']",
 "3.3191 : ['i2', 'i15']",
 "3.3191 : ['i15', 'i2']",
 "3.3191 : ['i2', 'i16']",
 "3.3191 : ['i16', 'i2']",
 "3.3191 : ['i2', 'i17']",
 "3.3191 : ['i17', 'i2']",
 "3.3191 : ['i2', 'i18']",
 "3.3191 : ['i18', 'i2']",
 "3.3268 : ['i2', 'i19']",
 "3.3350 : ['i19',

Although there is a ton of other cool stuff in the repo, this is all we wanted to really cover as it covers most of the more exciting parts and pieces of the search algorithm!  For more info on how search works, check out `src/scripts/search.py` for the script that handles the search algorithm and calls it.  Check out `src/search/search.py` for the actual search algorithm!